# 1. Проблема

![](Задача№3.jpeg)

# 2. Содержательная постановка задачи

Необходимо определить процентный состав шихты для выплавки стали, нормированной по ГОСТ'у и обладающей минимальной стоимостью материалов с учётом ограничений на кол-во материалов.

# 3. Формальная мат. модель

$C = c_1,c_2,c_3$ -  весы (цены) компонентов шихты для выплавки стали.  
$x = x_1,x_2,x_3$ - кол-во материалов(тонн) для выплавки стали.  
**Целевая функция** - минимальная стоимость материалов:  
$$ (C,x) = -\sum\limits_{i=1}^3c_ix_i  \rightarrow \max $$
$$x_1+x_2+x_3 \leq 200$$
$$0.15x_1 + 0.15x_2 + 0.17x_3 \geq 0.16(x_1+x_2+x_3)$$
$$0.15x_1 + 0.15x_2 + 0.17x_3 \leq 0.18(x_1+x_2+x_3)$$
$$0.10x_1 + 0.08x_2 + 0.09x_3 \leq 0.09(x_1+x_2+x_3)$$
$$0.007x_1 + 0.003x_2 + 0.005x_3 \leq 0.005(x_1+x_2+x_3)$$
$$0 \leq x_1 \leq 150$$
$$0 \leq x_2 \leq 100$$
$$0 \leq x_3 \leq 75$$


Запишем наши условия с помощью линейной алгебры:

$$
A = \begin{pmatrix}
  0.15 & 0.15 & 0.17\\
  0.1 & 0.08 & 0.09\\
  0.007 & 0.003 & 0.005\\
\end{pmatrix}
$$
$$\vec{x} = \begin{pmatrix}
x_1\\ x_2\\ x_3\\
\end{pmatrix}
$$
$$
\vec{B} = 
\begin{pmatrix}
0.16\\ 0.18\\ 0.09\\ 0.005
\end{pmatrix}
$$

# 4. Алгоритм и ПО

В качестве ПО будем использовать Python с подключенными модулями:
* numpy - для работы с линейной алгеброй
* cvxpy - для работы с линейным программированием

# 5. Решение задачи

In [1]:
import numpy as np
import cvxpy

In [2]:
def solution(A, b, c, stock):
    x = cvxpy.Variable(shape=len(c), integer = False)
    minimum = min(stock)
    constraints = [(A[0] @ x >= b[0]*sum(x)),
                   (A[0] @ x <= b[1]*sum(x)),
                   (A[1] @ x <= b[2]*sum(x)),
                   (A[2] @ x <= b[3]*sum(x)),
                   (sum(x) <= 200),
                   (sum(x) >= minimum),
                   (x >= 0),
                   (x[0] <= stock[0]),
                   (x[1] <= stock[1]),
                   (x[2] <= stock[2])]

    total_value =-(c @ x)

    problem = cvxpy.Problem(cvxpy.Maximize(total_value), constraints=constraints)
    print('Минимальная стоимость', -round(problem.solve(), 3))
    print('Кол-во тонн материалов', np.abs(np.round(x.value, 3)))
    #print(problem.solve())
    #print(x.value)

# 6. Анализ

Проверим наш алгоритм на реальных данных:

In [3]:
c = np.array([300,200,150]) # вектор стоимости
A = np.array([[0.15,0.15,0.17],
              [0.1,0.08,0.09],
              [0.007,0.003,0.005]]) # матрица кол-ва составов материалов
b = np.array([0.16, 0.18, 0.09, 0.005]) # вектор ограничений по ГОСТу
stock = np.array([150,100,75])

In [4]:
solution(A, b, c, stock)

Минимальная стоимость 11250.0
Кол-во тонн материалов [ 0.  0. 75.]


Как мы видим, выплавка производится только из 3 материала, т.к. только его состав подходит по ограничениям. Необходимо пересмотреть состав материалов, чтобы ограничения по ГОСТу выполнялись.